In [35]:
import pandas as pd
import json
import flair
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, Sentence
import torch
import re
import numpy as np

In [678]:
cuda = torch.device('cuda')
cpu = torch.device('cpu')

In [679]:
device = cpu

In [680]:
flair.device = device

In [681]:
emb = WordEmbeddings('pl')

In [917]:
with open("recipes_pretty.json") as file:
    data = json.load(file)

In [918]:
data

[{'link': ['https://kkpolska.pl/przepis/98/tropikalny_drink_bezalkoholowy_z_guawa_i_imbirem_.html'],
  'title': ['Tropikalny drink bezalkoholowy z guawą i imbirem '],
  'photo_link': ['images/przepisy/guawa_imbir.jpg'],
  'ingredients': [['3 łyżeczki soku z limonki \r'],
   ['1/2 szklanki napoju Rubicon Deluxe Guawa \r'],
   ['2 łyżki miodu \r'],
   ['2 łyżeczki imbiru ']],
  'preparation': ['Wszystkie składniki umieścić w blenderze i zmiksować. Tropikalny drink bardzo dobrze smakuje schłodzony. ']},
 {'link': ['https://kkpolska.pl/przepis/38/tropikalne_smoothie_z_lychee.html'],
  'title': ['Tropikalne smoothie z lychee'],
  'photo_link': ['images/przepisy/smoothie_lychee.jpg'],
  'ingredients': [['300ml jogurtu naturalnego 2%\r'],
   ['10 owoców lychee\r'],
   ['2 banany\r'],
   ['200g ananasa\r'],
   ['1 szklanka kruszonego lodu\r'],
   ['miód']],
  'preparation': ['1. Banany, lychee, ananasa pokroić na mniejsze kawałki.\r\n2. Owoce zalać jogurtem, dodać kruszony lód.\r\n3. Całość zm

# New Dataset

In [1022]:
with open("recipes3.json") as file:
    data = json.load(file)

In [1023]:
for d in data:
    d['ingredients'] = [ing for ing in d['ingredients'] if ing != '']
    for i, ing in enumerate(d['ingredients']):
        d['ingredients'][i] = [ing]

In [1024]:
data

[{'link': 'https://beszamel.se.pl/mielone/indycze-mielone-z-kukurydza-sprytny-przepis-na-pieczone-kotlety-z-indyka,22379/',
  'title': 'Indycze mielone z kukurydzą: sprytny przepis na pieczone kotlety z indyka',
  'portions': '4',
  'photo_link': 'https://cdn11.beszamel.smcloud.net/t/thumbs/660/441/1/user_photos/22426/indycze-mielone-z-kukurydza-sprytny-przepis-na-pi.jpg',
  'ingredients': [['500 g mielonego mięsa indyczego'],
   ['1 bułka'],
   ['1 cebula'],
   ['1 puszka kukurydzy konserwowej'],
   ['2 łyżki natki pietruszki, posiekanej'],
   ['sól i pieprz'],
   ['oraz do panierowania'],
   ['2 jajka rozmiar S rozmącone z 2 łyżkami wody'],
   ['mąka'],
   ['tarta bułka']],
  'preparation': 'Cebulę drobno posiekać. Bułkę namoczyć w ciepłej wodzie. Kukurydzę odsączyć.\xa0\nMięso mielone przełożyć do miski, dodać posiekaną cebulę, odciśniętą bułkę, przyprawić solą i pieprzem. Wymieszać, a następnie wyrabiać przez 2 minuty.\nDo masy mięsnej dodać odsączoną kukurydzę. Wymieszać na jednor

# Wyrazy kluczowe

In [922]:
temp = Sentence('jedzenie jednostka narzędzie')
emb.embed(temp)
jedzenie = temp[0].embedding
jednostka = temp[1].embedding
narzedzie = temp[2].embedding

# Przykładowy składnik

### Przygotowanie

In [923]:
ingredient = Sentence(data[0]["ingredients"][0][0])

In [924]:
emb.embed(ingredient)

[Sentence: "1 kg pieczarek pokrojonych wzdłuż na plasterki" - 7 Tokens]

In [925]:
for i in ingredient:
    print(i, i.embedding[:5])

Token: 1 1 tensor([ 0.2346,  0.2216,  0.0498, -0.7787,  0.3949])
Token: 2 kg tensor([ 0.5467,  0.3708,  0.1889, -0.9677,  0.4193])
Token: 3 pieczarek tensor([ 0.2236, -0.0007, -0.3181, -0.6170,  0.3877])
Token: 4 pokrojonych tensor([ 0.2358, -0.0906,  0.0420, -0.6892,  0.3711])
Token: 5 wzdłuż tensor([-0.3330, -0.3095,  0.2327, -0.3966,  0.3435])
Token: 6 na tensor([ 0.2342, -0.0429,  0.0537, -0.0767, -0.1966])
Token: 7 plasterki tensor([-0.1074,  0.3050, -0.1940, -1.3618,  0.4897])


### Odległość L2 od słowa "jedzenie"

In [926]:
for i in ingredient:
    print(torch.norm(i.embedding - jedzenie))

tensor(5.8111)
tensor(6.8939)
tensor(5.4464)
tensor(5.0615)
tensor(6.2661)
tensor(5.0469)
tensor(4.6938)


###  Podobieństwo cosinusowe od słowa jedzenie

In [927]:
def cos(v,u): return (v @ u)/v.norm()/u.norm()

In [928]:
for i in ingredient:
    if i.embedding.norm() > 0.001:
        print(i)
        print(cos(i.embedding, jedzenie))

Token: 1 1
tensor(0.1270)
Token: 2 kg
tensor(0.2780)
Token: 3 pieczarek
tensor(0.3626)
Token: 4 pokrojonych
tensor(0.3954)
Token: 5 wzdłuż
tensor(0.1010)
Token: 6 na
tensor(0.1897)
Token: 7 plasterki
tensor(0.4819)


###  Podobieństwo cosinusowe od słowa jednostka

In [929]:
for i in ingredient:
    if i.embedding.norm() > 0.001:
        print(i)
        print(cos(i.embedding, jednostka))

Token: 1 1
tensor(0.1691)
Token: 2 kg
tensor(0.1665)
Token: 3 pieczarek
tensor(0.0958)
Token: 4 pokrojonych
tensor(0.1662)
Token: 5 wzdłuż
tensor(0.2218)
Token: 6 na
tensor(0.1479)
Token: 7 plasterki
tensor(0.1248)


### Pod. do narzędzie

In [930]:
for i in ingredient:
    if i.embedding.norm() > 0.001:
        print(i)
        print(cos(i.embedding, narzedzie))

Token: 1 1
tensor(0.1972)
Token: 2 kg
tensor(0.0987)
Token: 3 pieczarek
tensor(0.2026)
Token: 4 pokrojonych
tensor(0.2689)
Token: 5 wzdłuż
tensor(0.0843)
Token: 6 na
tensor(0.1124)
Token: 7 plasterki
tensor(0.2210)


In [931]:
for d in data:
    for i in d['ingredients']:
        print(i[0].split())

['1', 'kg', 'pieczarek', 'pokrojonych', 'wzdłuż', 'na', 'plasterki']
['2', 'łyżki', 'octu', 'balsamicznego']
['1', 'łyżka', 'sosu', 'sojowego']
['3', 'ząbki', 'czosnku,', 'posiekane']
['1/2', 'łyżeczki', 'tymianku', 'suszonego']
['sól', 'i', 'pieprz', 'do', 'smaku']
['1', 'łyżka', 'natki', 'pietruszki', 'posiekanej']
['2', 'szklanki', 'pąków', 'mniszka', 'lekarskiego']
['2/3', 'szklanki', 'octu']
['1/3', 'szklanki', 'wody']
['1', 'łyżeczka', 'soli']
['200', 'g', 'pomidorków', 'koktajlowych', '(ewentualnie', 'innych', 'pomidorów)']
['1', 'opakowanie', 'młodych', 'liści']
['5', 'świeżych', 'pokrojonych', 'w', 'cząstki']
['kilka', 'listków', 'świeżej', 'bazylii']
['czerwona', 'cebula', 'pokrojona', 'w', 'plastry']
['2-3', 'łyżki', 'oliwy', 'z', 'oliwek']
['1-2', 'łyżki', 'octu', 'balsamicznego']
['szczypta', 'soli', 'morskiej', 'do', 'smaku']
['1', 'duży', 'pęczek', 'buraków', '(około', '4-6', 'buraków)', 'z', 'bujnymi', 'zielonymi', 'liśćmi']
['1', 'łyżka', 'oliwy', 'z', 'oliwek']
['2', 

['pieprz']
['2', 'łyżki', 'cukru']
['3', 'łyżki', 'octu', 'spirytusowego']
['2', 'cebulki', 'szalotki']
['1', 'łyżeczka', 'mąki', 'ziemniaczanej']
['łyżka', 'posiekanej', 'kolendry']
['200', 'g', 'tofu']
['2', 'porcje', 'rukoli']
['1', 'awokado']
['sól']
['świeżo', 'zmielony', 'pieprz']
['oliwa', 'extra', 'virgine']
['sok', 'z', 'cytryny']
['2', 'łyżki', 'masła']
['3', 'jajka']
['1', 'łyżka', 'posiekanego', 'szczypiorku']
['grzanki', 'z', 'chleba', 'żytniego', 'posmarowane', 'masłem']
['340', 'g', 'mąki', 'z', 'orzechów', '(nerkowce', 'i', 'migdały)']
['1', 'łyżka', 'cynamonu', 'cejlońskiego']
['2', 'łyżeczki', 'sody', 'oczyszczonej', '/', 'lub', 'proszku', 'do', 'pieczenia']
['szczypta', 'soli', 'himalajskiej', '/', 'lub', 'morskiej']
['20-30', 'g', 'cukru', 'palmowego']
['175', 'g', 'musu', 'jabłkowego']
['50', 'g', 'oleju', 'rzepakowego']
['2', 'jajka']
['1', 'łyżeczka', 'cukru', 'waniliowego']
['2', 'ćwiartki', 'z', 'kurczaka']
['500', 'g', 'ryżu', 'kleistego', 'Q', 'Rice']
['15', 

['2', 'duże', 'ząbki', 'czosnku']
['1', 'papryczka', 'chili']
['1', 'łyżka', 'oleju', 'kokosowego']
['3', 'łyżki', 'sezamu']
['0,5', 'szklanki', 'listków', 'mięty']
['1', 'łyżka', 'liści', 'kolendry']
['Sos:', '•', 'sok', 'z', '2', 'limonek']
['2', 'łyżki', 'cukru', 'trzcinowego']
['1', 'łyżka', 'sosu', 'ostrygowego', '/', 'lub', '2', 'łyżki', 'sojowego']
['Chińska', 'mieszanka', 'warzyw', '330', 'g']
['2', 'filety', 'z', 'piersi', 'kaczki']
['250', 'chińskiego', 'makaronu', 'jajecznego']
['2', 'łyżki', 'sosu', 'sojowego']
['2', 'łyżki', 'octu']
['2', 'łyżki', 'oleju']
['1', 'łyżeczka', 'cukru']
['szczypta', 'imbiru', 'w', 'proszku']
['szczypta', 'ostrej', 'papryki']
['szczypta', 'słodkiej', 'papryki']
['szczypta', 'gałki', 'muszkatołowej']
['szczypta', 'cynamonu']
['sól']
['pieprz']
['80', 'g', 'makaronu', 'Chow', 'Mein']
['170', 'g', 'wołowiny']
['1', 'ząbek', 'czosnku']
['1', 'łyżka', 'startego', 'świeżego', 'imbiru']
['1/4', 'pora']
['1/2', 'czerwonej', 'papryki']
['3', 'pieczarki'

['sól', 'i', 'pieprz']
['około', '1,5', 'szklanki', 'truskawek']
['150', 'g', 'makaronu', 'chilli', 'Bali', 'Kitchen']
['1', 'łagodna', 'czerwona', 'papryczka', 'chili,', 'bez', 'pestek,', 'posiekana']
['2', 'łyżki', 'masła']
['1/3', 'szklanki', 'śmietanki', 'kremowej', '30%']
['0,5', 'szklanki', 'drobno', 'tartego', 'parmezanu']
['0,5', 'szklanki', 'liści', 'bazylii', 'i', 'mięty']
['1', 'łyżka', 'sosu', 'Worcester']
['80', 'g', 'makaronu', 'chili']
['170', 'g', 'wołowiny']
['1', 'ząbek', 'czosnku']
['1', 'łyżka', 'startego', 'świeżego', 'imbiru']
['1/4', 'pora']
['1/2', 'czerwonej', 'papryki']
['3', 'pieczarki']
['2', 'łyżki', 'oleju', 'roślinnego']
['szczypta', 'zmielonego', 'białego', 'pieprzu']
['szczypta', 'cukru']
['3', 'łyżki', 'ciemnego', 'sosu', 'sojowego', '+', '2', 'łyżki', 'wody']
['2', 'łyżeczki', 'oleju', 'sezamowego']
['2', 'łyżki', 'posiekanego', 'szczypiorku']
['80', 'g', 'makaronu', 'pszennego', 'z', 'czarnym', 'sezamem']
['2', 'pomidory', 'malinowe']
['miseczka', 'k

['2', 'małe', 'bataty']
['1', 'ząbek', 'czosnku']
['150', 'g', 'ciecierzycy']
['oliwa', 'extra', 'virgin']
['80', 'g', 'szpinaku']
['100', 'g', 'fety']
['2', 'łyżki', 'krojonych', 'pomidorów', 'z', 'puszki', '/', 'lub', '1/2', 'świeżego', 'pomidora']
['papryczka', 'chili']
['sól']
['pieprz']
['1', 'łyżeczka', 'kminu', 'rzymskiego']
['1/2', 'łyżeczki', 'suszonego', 'oregano']
['1/2', 'łyżeczki', 'czerwonej', 'papryki', 'słodkiej']
['1/3', 'łyżeczki', 'papryki', 'ostrej']
['1/3', 'łyżeczki', 'papryki', 'wędzonej']
['świeża', 'kolendra']
['jogurt', 'naturalny']
['4', 'filety', 'z', 'łososia', '(każdy', 'filet', 'po', 'około', '100', '-', '150', 'g)']
['pasta', 'harissa']
['oliwa', 'z', 'oliwek']
['sól']
['400-500', 'g', 'ciecierzycy']
['1', 'łyżka', 'oliwy', 'extra', 'virgin']
['1', 'cebula']
['2', 'ząbki', 'czosnku']
['1', 'łyżka', 'oleju', 'kokosowego', '/', 'lub', 'masła']
['2', 'ziemniaki']
['1', 'łyżeczka', 'mielonego', 'kminu', 'rzymskiego']
['500', 'ml', 'bulionu', 'lub', 'rosołu']

['Dodatki:', '•', '200', 'g', 'świeżego', '(najlepiej', 'surowego', 'lub', 'wędzonego)', 'filetu', 'z', 'łososia']
['dojrzały', 'owoc', 'awokadu']
['pęczek', 'szczypiorku']
['marynowany', 'imbir', '(ze', 'słoika)', 'według', 'uznania']
['pasta', 'wasabi']
['200', 'g', 'filetu', 'z', 'łososia', 'bez', 'skóry', 'i', 'ości']
['100', 'g', 'makaronu', 'ryżowego']
['2', 'łyżki', 'oleju', 'roślinnego']
['1', 'papryczka', 'chili']
['1,5', 'łyżki', 'startego', 'imbiru']
['2', 'łyżki', 'sezamu']
['2', 'łyżki', 'pasty', 'miso']
['2', 'łyżki', 'sosu', 'sojowego']
['1', 'limonka']
['1/2', 'łyżeczki', 'cukru']
['1', 'łyżka', 'oleju', 'sezamowego']
['4', 'łyżki', 'posiekanego', 'szczypiorku']
['świeża', 'kolendra']
['Tempura', 'Samlip']
['olej', 'kokosowy']
['surowe', 'krewetki', '24', 'szt.']
['mąka', 'pszenna', '1', 'szklanka']
['sól']
['woda']
['Zupa', 'Tom', 'Yum', 'THAI', 'PRIDE']
['Pędy', 'bambusa', 'THAI', 'PRIDE']
['Grzyby', 'Shiitake', 'DIAMOND']
['Makaron', 'ryżowy', '3', 'mm', 'FARMER']
['

['3-4', 'łyżki', 'oleju']
['kilka', 'listków', 'bazylii']
['200', 'g', 'makaronu', 'ryżowego']
['2', 'ząbki', 'czosnku']
['1', 'papryczka', 'chili']
['3-4', 'łyżki', 'sosu', 'sojowego']
['1', 'łyżka', 'octu', 'ryżowego']
['garść', 'posiekanej', 'kolendry']
['4', 'kawałki', 'mięsa', 'z', 'udek', 'z', 'kurczaka', '(bez', 'skóry', 'i', 'kości)']
['sól']
['pieprz']
['przyprawa', '5', 'smaków']
['słodki', 'sos', 'chili', '(1-2', 'łyżki', 'stołowe)']
['sezam', 'biały']
['kolendra', 'do', 'dekoracji', '(opcjonalnie)']
['Składniki', 'na', 'pudding:', '•', '800', 'ml', 'mleczka', 'kokosowego']
['1/3', 'szklanki', 'tapioki']
['1', 'łyżka', 'cukru', 'trzcinowego']
['szczypta', 'soli']
['Składniki', 'na', 'mus', 'truskawkowy:', '•', '2', 'szklanki', 'truskawek']
['1', 'łyżka', 'cukru', 'trzcinowego']
['sok', 'z', 'połowy', 'cytryny']
['2', 'podwójne', 'filety', 'z', 'kurczaka']
['2', 'łyżki', 'pasty', 'Tom', 'Yum', 'Thai', 'Pride']
['sos', 'sojowy', 'Lee', 'Kum', 'Kee']
['sos', 'rybny', 'Cock']
['

['1', 'garść', 'kiełków', 'fasoli', 'mung']
['1', 'łyżka', 'mielonej', 'trawy', 'cytrynowej']
['2-3', 'łyżki', 'sosu', 'teryiaki']
['olej', 'arachidowy']
['1', 'łyżka', 'sosu', 'rybnego']
['1', 'czerwona', 'cebula']
['2', 'łyżki', 'sosu', 'ostrygowego']
['1', 'łyżka', 'octu', 'ryżowego']
['świeża', 'kolendra']
['800', 'ml', 'mleczka', 'koksowego']
['¾', 'szklanki', 'tapioki']
['1', 'laska', 'wanilii']
['3-4', 'łyżki', 'cukru']
['2', 'płaskie', 'łyżeczki', 'agaru']
['szczypta', 'soli', 'himalajskiej', 'lub', 'morskiej']
['100', 'g', 'gorzkiej', 'czekolady', '90%']
['1', 'łyżeczka', 'oleju', 'kokosowego']
['1', 'łyżka', 'oleju', 'kokosowego']
['1', 'ząbek', 'czosnku']
['1/2', 'cebuli', 'cukrowej']
['łodyga', 'selera', 'naciowego', 'lub', 'pora']
['500', 'g', 'marchewki']
['2', 'łyżeczki', 'startego', 'imbiru']
['1', 'łyżka', 'masła', 'orzechowego']
['1/2', 'łyżeczki', 'chili', 'w', 'proszku']
['1/2', 'łyżeczki', 'kurkumy']
['1/2', 'łyżeczki', 'kminu', 'rzymskiego']
['szczypta', 'cynamonu

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Proof of concept

In [932]:
%run words.py

In [933]:
nie_jednostki = [
    'jogurtu',
    'banany',
    'ostrej',
    'czerwonej',
    'piersi',
    'szparagów',
    'kapustka',
    'cukier',
    'świeżego',
    'dojrzałe',
    'Rumu',
    'dojrzałe',
    'czystej',
    'smakowe',
    'tequili',
    'limonki',
    'fasoli',
    'żeberek',
    "truskawek",
    'mąki',
    'pokrojonego',
    'cienkie',
    'cebuli',
    'kukurydzy',
    'bambusa',
    'posiekanych',
    'oleju',
    'oliwek',
    'papryczka',
    'szczypiorek',
    'kiełków',
    'orzeszków',
    'trawy',
    'ziemniaków',
    'pomidorów',
    'moreli',
    'mleczka',
    'wódki',
    'koktajlowe'
]

In [934]:
jednostki = Sentence(" ".join(jednostki))
nie_jednostki = Sentence(" ".join(nie_jednostki))
emb.embed(jednostki)
emb.embed(nie_jednostki)

[Sentence: "jogurtu banany ostrej czerwonej piersi szparagów kapustka cukier świeżego dojrzałe Rumu dojrzałe czystej smakowe tequili limonki fasoli żeberek truskawek mąki pokrojonego cienkie cebuli kukurydzy bambusa posiekanych oleju oliwek papryczka szczypiorek kiełków orzeszków trawy ziemniaków pomidorów moreli mleczka wódki koktajlowe" - 39 Tokens]

In [935]:
X_jed = torch.cat([j.embedding for j in jednostki]).reshape(len(jednostki),300)
X_njed = torch.cat([j.embedding for j in nie_jednostki]).reshape(len(nie_jednostki),300)

In [968]:
def knn(Y):
    mask = Y.norm(dim=1) != 0
    if not mask.any(): return torch.BoolTensor(mask)
    
    cos_sim_jed = X_jed @ Y[mask,:].T / (X_jed.norm(dim=1, keepdim=True) * Y[mask,:].norm(dim=1, keepdim=True).T)
    cos_sim_njed = X_njed @ Y[mask,:].T / (X_njed.norm(dim=1, keepdim=True) * Y[mask,:].norm(dim=1, keepdim=True).T)

    ret = torch.zeros(Y.shape[0], dtype=torch.bool).to(device)
    ret[mask] = (cos_sim_jed.max(0)[0] > cos_sim_njed.max(0)[0]) & (cos_sim_jed.max(0)[0] > 0.5)
    return ret

In [969]:
def sentenceToMat(s):
    s = Sentence(s)
    emb.embed(s)
    r = torch.cat([i.embedding for i in s])
    return r.reshape(int(r.shape[0]/300), 300), s

In [970]:
knn(sentenceToMat('1 łyżka bezalkoholowy z guawą i imbirem mega')[0])

tensor([False,  True, False, False, False, False, False, False])

In [971]:
knn(sentenceToMat('234 Łyżeczek kefiru')[0])

tensor([False,  True, False])

In [972]:
knn(sentenceToMat('rabarbar 4 łodygi')[0])

tensor([False, False,  True])

In [973]:
Unit = []
for d in data:
    rec = []
    for i in d['ingredients']:
        mat, s = sentenceToMat(i[0])
        mask = knn(mat)
        rec.append([s[i] for i in range(len(s)) if mask[i]])
    Unit.append(rec)

In [974]:
Units_per_recipe = [[" ".join([u.text for u in units]) for units in U] for U in Unit] 

In [975]:
Unit

[[[Token: 7 plasterki],
  [Token: 2 łyżki],
  [Token: 2 łyżka],
  [Token: 2 ząbki],
  [Token: 2 łyżeczki],
  [],
  [Token: 2 łyżka]],
 [[Token: 2 szklanki, Token: 3 pąków],
  [Token: 2 szklanki],
  [Token: 2 szklanki],
  [Token: 2 łyżeczka]],
 [[Token: 2 g],
  [Token: 2 opakowanie, Token: 4 liści],
  [Token: 5 cząstki],
  [Token: 2 listków],
  [Token: 5 plastry],
  [Token: 2 łyżki],
  [Token: 2 łyżki],
  [Token: 1 szczypta]],
 [[Token: 11 liśćmi],
  [Token: 2 łyżka],
  [Token: 3 ząbki],
  [Token: 2 łyżeczka, Token: 4 soku],
  [],
  []],
 [[Token: 2 g],
  [Token: 2 g],
  [],
  [Token: 2 łyżek],
  [Token: 2 szklanki],
  [Token: 2 łyżek],
  [],
  [Token: 2 g],
  [Token: 2 łyżki]],
 [[Token: 2 g],
  [Token: 2 ząbki],
  [],
  [Token: 2 opakowanie],
  [Token: 2 łyżki],
  [Token: 2 łyżki, Token: 3 soku],
  [Token: 2 łyżka],
  [],
  []],
 [[],
  [],
  [Token: 2 ząbki],
  [Token: 2 g, Token: 5 liściach],
  [],
  [Token: 1 szczypta],
  [],
  [],
  [],
  []],
 [[Token: 2 g, Token: 4 liści], [], [

# Full

In [976]:
przyprawy, warzywa, nabial, zboza, tluszcze, miesa;

NameError: name 'miesa' is not defined

In [977]:
def knn(Y, X_jed, X_njed, d):
    mask = Y.norm(dim=1) != 0
    if not mask.any(): return torch.BoolTensor(mask)
    
    cos_sim_jed = X_jed @ Y[mask,:].T / (X_jed.norm(dim=1, keepdim=True) * Y[mask,:].norm(dim=1, keepdim=True).T)
    cos_sim_njed = X_njed @ Y[mask,:].T / (X_njed.norm(dim=1, keepdim=True) * Y[mask,:].norm(dim=1, keepdim=True).T)

    ret = torch.zeros(Y.shape[0], dtype=torch.bool).to(device)
    
    ret[mask] = (d*cos_sim_jed.max(0)[0] > cos_sim_njed.max(0)[0]) & (cos_sim_jed.max(0)[0] > 0.5)
    return ret

In [978]:
def gen_knn(x, nx, d):
    x = Sentence(" ".join(x))
    nx = Sentence(" ".join(nx))
    emb.embed(x)
    emb.embed(nx)
    x = [xx for xx in x if xx.embedding.norm() > 0.001]
    nx = [xx for xx in nx if xx.embedding.norm() > 0.001]
    X_x = torch.cat([j.embedding for j in x]).reshape(len(x),300)
    X_nx = torch.cat([j.embedding for j in nx]).reshape(len(nx),300)
    return lambda Y: knn(Y, X_x, X_nx, d)

In [979]:
s = Sentence("lód asdf piwo")

In [980]:
emb.embed(s)

[Sentence: "lód asdf piwo" - 3 Tokens]

In [981]:
[sss.embedding for sss in [ss for ss in s if ss.embedding.norm() > 0.001]]

[tensor([-2.8112e-01, -5.0371e-02,  4.8964e-02, -6.7958e-01,  7.4694e-01,
          2.0344e-01,  1.2213e-01, -1.9825e-01,  1.3946e-01,  5.3816e-01,
          4.6904e-01,  9.8772e-02,  4.3799e-01,  2.9822e-01, -1.3622e-01,
          5.4849e-01,  2.3856e-01,  2.7953e-01,  5.4909e-02, -1.3054e-01,
         -3.9186e-01, -8.3339e-02, -6.2480e-01, -1.0207e-02, -4.2332e-01,
          2.4943e-01, -2.0429e-01, -9.1938e-02, -2.8870e-01, -2.6780e-02,
         -1.8569e-01, -1.5805e-01,  2.6072e-01, -2.9521e-01,  4.3091e-01,
          5.5229e-01,  2.2313e-02,  2.3725e-01, -1.7151e-01, -9.3433e-04,
         -3.3377e-01,  4.8355e-02, -1.1487e-01, -2.0072e-01,  2.6008e-01,
         -8.1873e-01,  1.1706e-01, -2.3152e-01, -1.1688e-01, -1.7142e-01,
          3.1835e-01, -8.2847e-02,  5.8493e-01,  3.5867e-01,  4.6979e-02,
         -1.8964e-01, -1.6925e-01, -6.0271e-01,  4.0612e-01,  1.6662e-01,
         -9.4596e-02, -2.0708e-01,  3.1231e-01, -1.7679e-01,  2.4070e-01,
          6.0691e-02,  6.7991e-01,  1.

In [982]:
def predict(knn, data):
    Unit = []
    for d in data:
        rec = []
        for i in d['ingredients']:
            mat, s = sentenceToMat(i[0])
            mask = knn(mat)
            rec.append([s[i] for i in range(len(s)) if mask[i]])
        Unit.append(rec)
        
    return [[" ".join([u.text for u in units]) for units in U] for U in Unit] 

# Jednostki

In [983]:
%run words.py

In [984]:
knn_jednostki = gen_knn(jednostki, przyprawy + warzywa + nabial + zboza + tluszcze, 1)

In [985]:
predict(knn_jednostki, data)

[['plasterki', 'łyżki', 'łyżka', 'ząbki', 'łyżeczki', '', 'łyżka'],
 ['szklanki pąków', 'szklanki', 'szklanki', 'łyżeczka'],
 ['g',
  'opakowanie liści',
  'cząstki',
  'listków',
  'plastry',
  'łyżki oliwek',
  'łyżki',
  'szczypta'],
 ['liśćmi', 'łyżka oliwek', 'ząbki', 'łyżeczka soku', '', ''],
 ['g truskawek', 'g', '', 'łyżek', 'szklanki', 'łyżek', '', 'g', 'łyżki'],
 ['g mięsa',
  'ząbki',
  '',
  'opakowanie',
  'łyżki',
  'łyżki soku',
  'łyżka',
  '',
  ''],
 ['', '', 'ząbki', 'g liściach', '', 'szczypta', '', '', '', ''],
 ['g liści', '', 'l', '', ''],
 ['opakowania',
  '',
  '',
  '',
  '',
  '',
  'łyżek oliwek',
  'łyżeczki',
  'łyżeczki',
  ''],
 ['g',
  'g zdrewniałych kawałki',
  'łyżka',
  'g plastry',
  'plasterki',
  'łyżeczka',
  'ml',
  'ml',
  'g',
  'łyżeczki soku',
  ''],
 ['', '', '', 'litra'],
 ['', 'łyżeczki', 'szklanka', 'truskawek', ''],
 ['', 'ząbki', 'łyżka', 'łyżki', 'łyżeczka', 'szklanki liści', '', ''],
 ['',
  '',
  'ząbki',
  'łyżka',
  'łyżka',
  's

In [986]:
Sentence('• 2 łyżki stołowe sosu do pierożków Lee Kum Kee')

Sentence: "• 2 łyżki stołowe sosu do pierożków Lee Kum Kee" - 10 Tokens

In [987]:
nie_przyprawy = ['stołowa']

In [988]:
knn_przyprawy = gen_knn(przyprawy, jednostki + warzywa + nabial + zboza + tluszcze + nie_przyprawy, 1)

In [989]:
predict(knn_przyprawy, data)

[['', 'balsamicznego', '', '', 'tymianku', 'sól pieprz smaku', ''],
 ['', '', '', ''],
 ['', '', '', 'bazylii', '', '', 'balsamicznego', 'smaku'],
 ['', '', '', 'cytryny', 'sól', 'pieprz'],
 ['', '', '', '', 'cukru', '', 'mascarpone', '', 'cukru waniliowego'],
 ['', '', '', '', '', 'cytryny', '', 'pieprz sól', 'koperku'],
 ['', '', '', '', 'gałka muszkatołowa', 'pieprzu', 'sól', '', '', ''],
 ['', '', '', '', 'pieprz'],
 ['', '', 'granatu', '', 'migdałów', '', '', '', 'balsamicznego', 'pieprz'],
 ['', 'bez', '', '', '', '', '', '', '', 'cytryny smaku', 'sól pieprz'],
 ['', 'cytryny', 'cukru', ''],
 ['', '', 'cukru', '', 'melisy mięty'],
 ['', '', '', '', 'kminku', '', 'sól pieprz', ''],
 ['', '', '', '', '', '', '', 'cytryny smaku', 'sól', '', ''],
 ['',
  '',
  '',
  '',
  '',
  '',
  'mięty pieprzowej',
  '',
  '',
  'sól pieprz',
  'sos',
  '',
  '',
  'oregano',
  'mięty pieprzowej',
  'cukru',
  ''],
 ['', '', '', 'słodki', '', '', 'imbiru', '', 'curry', 'sól pieprz', '', ''],
 ['

In [990]:
nie_warzywa = ['zupy', 'sosów', 'małży', 'smażonej', 'marynowanych', 'musztardy', 'chleba', 'świeże', 'ugotowanych', 'czarny', 'japońskiej']

In [991]:
knn_warzywa = gen_knn(warzywa, jednostki + przyprawy + nabial + zboza + tluszcze + nie_warzywa, 1)

In [992]:
predict(knn_warzywa, data)

[['pieczarek', '', '', '', '', '', ''],
 ['', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', ''],
 ['', 'rabarbaru', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', 'pomidorowy', '', 'pomidory'],
 ['', 'ziemniaków', 'warzywnego', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', '', '', 'pieczarek', '', '', 'warzywnego', '', '', '', ''],
 ['', '', '', ''],
 ['', '', '', '', ''],
 ['cukinie', '', 'pomidorowego', '', '', '', '', ''],
 ['', '', '', '', '', 'warzywnego', '', 'buraczanego', '', '', ''],
 ['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'pomidorowego',
  '',
  '',
  '',
  '',
  ''],
 ['soczewicy',
  'pomidorowego',
  'warzywnego',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['baldachów', '', 'skórka', 'płaskie', '', '']

In [1013]:
nie_nabial = ['Napój', 'cukier', 'gazowanego', 'w', 'pasty', 'groszku', 'bulionu', 'kleistego', 'koktajlowych']

In [1016]:
knn_nabial = gen_knn(nabial, jednostki + przyprawy + warzywa + zboza + tluszcze + nie_nabial, 0.9)

In [1017]:
predict(knn_nabial, data)

[['', '', '', '', '', '', ''],
 ['', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', ''],
 ['', '', '', 'śmietankowego', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', '', '', 'śmietanka', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', 'śmietany', '', '', ''],
 ['', '', '', ''],
 ['', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', 'śmietany', ''],
 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', 'mleka', ''],
 ['',
  'mleka',
  'ser',
  'mleka skondensowanego',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['', '', 'mleka', '', '', 'proszku', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', ''],
 ['', '', '', '', ''],
 ['', 'mleka', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', 'spr

In [1018]:
knn_zboza = gen_knn(zboza, jednostki + przyprawy + warzywa + nabial + tluszcze, 0.7)

In [1019]:
predict(knn_zboza, data)

[['kg', '', '', '', '', '', ''],
 ['', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['bujnymi', '', '', '', '', ''],
 ['', '', '', '', '', '', 'kg', '', ''],
 ['', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['makaronu', '', '', '', '', '', '', '', '', '', ''],
 ['', '', 'kg', ''],
 ['', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', 'mąka', ''],
 ['', '', '', '', '', ''],
 ['', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', ''

In [998]:
knn_tluszcze = gen_knn(tluszcze, zboza + jednostki + przyprawy + warzywa + nabial, 0.8)

In [999]:
predict(knn_tluszcze, data)

[['', '', '', '', '', '', ''],
 ['lekarskiego', '', '', ''],
 ['', '', '', '', '', 'oliwy', '', ''],
 ['', 'oliwy', '', '', '', ''],
 ['', '', '', '', '', '', '', '', ''],
 ['', '', '', '', 'oliwy', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', ''],
 ['', '', '', 'awokado', 'laskowych', '', 'oliwy', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', ''],
 ['', '', '', '', ''],
 ['', '', '', '', '', '', '', 'oliwa'],
 ['', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', 'tłuszcz', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', ''],
 ['', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', 'oliwie', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', ''],
 ['', '', 'awokado', '', ''

In [1009]:
knn_skladnik = gen_knn(tluszcze + zboza + przyprawy + warzywa + nabial, jednostki, 1)

In [1010]:
predict(knn_skladnik, data)

[['kg pieczarek pokrojonych',
  'octu balsamicznego',
  'sosu sojowego',
  'posiekane',
  'tymianku suszonego',
  'sól pieprz smaku',
  'natki pietruszki posiekanej'],
 ['mniszka lekarskiego', 'octu', '', 'soli'],
 ['koktajlowych',
  '',
  'świeżych pokrojonych',
  'bazylii',
  'czerwona cebula pokrojona',
  'oliwy',
  'octu balsamicznego',
  'soli smaku'],
 ['pęczek buraków bujnymi zielonymi',
  'oliwy',
  'krojone mielone',
  'cytryny',
  'sól',
  'zmielony pieprz'],
 ['',
  'rabarbaru',
  'biszkoptów',
  'syropu truskawkowego',
  'cukru',
  'likieru pomarańczowego',
  'kg mascarpone',
  'twarogu',
  'cukru waniliowego'],
 ['wędzonego',
  'czosnku',
  'jajka',
  'śmietankowego',
  'oliwy',
  'cytryny',
  'mielonej słodkiej papryki',
  'pieprz sól',
  'koperku'],
 ['',
  'cebule',
  '',
  'mrożonego',
  'gałka muszkatołowa',
  'pieprzu',
  'sól',
  'koncentrat pomidorowy',
  'majonez',
  'suszone pomidory oleju'],
 ['świeżych', 'ziemniaków', 'bulionu warzywnego', 'śmietanka zup', 'pie

In [1002]:
[d['ingredients'] for d in data]

[[['1 kg pieczarek pokrojonych wzdłuż na plasterki'],
  ['2 łyżki octu balsamicznego'],
  ['1 łyżka sosu sojowego'],
  ['3 ząbki czosnku, posiekane'],
  ['1/2 łyżeczki tymianku suszonego'],
  ['sól i pieprz do smaku'],
  ['1 łyżka natki pietruszki posiekanej']],
 [['2 szklanki pąków mniszka lekarskiego'],
  ['2/3 szklanki octu'],
  ['1/3 szklanki wody'],
  ['1 łyżeczka soli']],
 [['200 g pomidorków koktajlowych (ewentualnie innych pomidorów)'],
  ['1 opakowanie młodych liści'],
  ['5\xa0 świeżych  pokrojonych w cząstki'],
  ['kilka listków świeżej bazylii'],
  ['czerwona cebula pokrojona w plastry'],
  ['2-3 łyżki oliwy z oliwek'],
  ['1-2 łyżki octu balsamicznego'],
  ['szczypta soli morskiej do smaku']],
 [['1 duży pęczek buraków (około 4-6 buraków) z bujnymi zielonymi liśćmi'],
  ['1 łyżka oliwy z oliwek'],
  ['2 duże ząbki czosnku, cienko krojone lub mielone'],
  ['1 łyżeczka świeżego soku z cytryny'],
  ['sól morska'],
  ['świeżo zmielony pieprz']],
 [['250 g truskawek'],
  ['250 

In [1020]:
data

[{'link': 'https://beszamel.se.pl/weganskie/balsamiczne-szaszlyki-z-pieczarek-pyszne-z-grilla-lub-piekarnika,22240/',
  'title': 'Balsamiczne szaszłyki z pieczarek: pyszne z grilla lub piekarnika',
  'portions': '4',
  'photo_link': 'https://cdn26.beszamel.smcloud.net/t/thumbs/660/441/1/user_photos/22241/balsamiczne-szaszlyki-z-pieczarek-pyszne-z-grilla.jpg',
  'ingredients': [['1 kg pieczarek pokrojonych wzdłuż na plasterki'],
   ['2 łyżki octu balsamicznego'],
   ['1 łyżka sosu sojowego'],
   ['3 ząbki czosnku, posiekane'],
   ['1/2 łyżeczki tymianku suszonego'],
   ['sól i pieprz do smaku'],
   ['1 łyżka natki pietruszki posiekanej']],
  'preparation': 'Z podanych składników przygotowujemy marynatę.\nMarynujemy grzyby w marynacie przez 30 minut.\nGrzyby nadziać wzdłuż (od kapelusza do nóżki) na patyczki do szaszłyków.\nSzaszłyki pieczemy w temperaturze 180°C lub\xa0grillujemy na średnim ogniu, aż będą miękkie i lekko zrumienione, około 2-3 minuty z każdej strony.\nPrzed podaniem pos